In [15]:
import requests
import pandas as pd
from fredapi import Fred

fred = Fred(api_key = 'fd0c3925530e1adec9762968a731218b')
api_key = ('fd0c3925530e1adec9762968a731218b')

In [63]:
import requests
import pandas as pd

# Your FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# SP500 % MoM Change
series_id_sp500 = 'SPASTT01USM661N'
units_sp500 = 'pch'             # Percent change from previous period
frequency_sp500 = 'm'           # Monthly frequency
start_date_sp500 = '1976-06-01'
end_date_sp500 = '2025-03-27'

# URL and parameters
url_sp500 = "https://api.stlouisfed.org/fred/series/observations"
params_sp500 = {
    'series_id': series_id_sp500,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_sp500,
    'frequency': frequency_sp500,
    'observation_start': start_date_sp500,
    'observation_end': end_date_sp500
}

# Make the request
response_sp500 = requests.get(url_sp500, params=params_sp500)

# Format response
if response_sp500.status_code == 200:
    data_sp500 = response_sp500.json()
    df_sp500 = pd.DataFrame(data_sp500['observations'])

    df_sp500['date'] = pd.to_datetime(df_sp500['date'])
    df_sp500['value'] = pd.to_numeric(df_sp500['value'], errors='coerce')
    df_sp500 = df_sp500[['date', 'value']].set_index('date')
    df_sp500 = df_sp500.dropna()

    df_sp500.rename(columns={'value': 'S&P 500 MoM Return (%)'}, inplace=True)
    df_sp500.index.name = "Date"

    print(df_sp500.head(10))

else:
    print('Failed to retrieve S&P 500 data.')
    print('Status Code:', response_sp500.status_code)
    print(response_sp500.text)


            S&P 500 MoM Return (%)
Date                              
1976-06-01                 0.66043
1976-07-01                 2.66918
1976-08-01                -0.88714
1976-09-01                 2.01072
1976-10-01                -3.29995
1976-11-01                -0.48088
1976-12-01                 3.99625
1977-01-01                -0.09331
1977-02-01                -2.41380
1977-03-01                -0.46798


In [65]:
import requests
import pandas as pd

# Your FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# Series details for GS10 (10-Year Treasury Yield)
series_id_bonds = 'GS10'
units_bonds = 'lin'                   # Raw yield values (%)
frequency_bonds = 'm'                 # Monthly frequency
start_date_bonds = '1976-06-01'
end_date_bonds = '2025-03-27'

# FRED API URL and parameters
url_bonds = "https://api.stlouisfed.org/fred/series/observations"
params_bonds = {
    'series_id': series_id_bonds,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_bonds,
    'frequency': frequency_bonds,
    'observation_start': start_date_bonds,
    'observation_end': end_date_bonds
}

# Make the API request
response_bonds = requests.get(url_bonds, params=params_bonds)

# Format and process the response
if response_bonds.status_code == 200:
    data_bonds = response_bonds.json()['observations']
    df_bonds = pd.DataFrame(data_bonds)

    # Convert and clean
    df_bonds['date'] = pd.to_datetime(df_bonds['date'])
    df_bonds['value'] = pd.to_numeric(df_bonds['value'], errors='coerce')
    df_bonds = df_bonds[['date', 'value']].dropna().set_index('date')
    df_bonds.rename(columns={'value': '10Y Treasury Yield (%)'}, inplace=True)

    # Calculate yield change and estimate return
    df_bonds['Yield Change'] = df_bonds['10Y Treasury Yield (%)'].diff()
    duration = 9  # Assumed duration for 10-year Treasuries

    df_bonds['Bond Return (%)'] = (
        df_bonds['10Y Treasury Yield (%)'] / 12
        - duration * df_bonds['Yield Change']
    )

    # Final cleanup
    df_bonds = df_bonds.dropna()
    df_bonds.index.name = "Date"

    # Preview
    print(df_bonds.head())

else:
    print('Failed to retrieve GS10 data.')
    print('Status Code:', response_bonds.status_code)
    print(response_bonds.text)



            10Y Treasury Yield (%)  Yield Change  Bond Return (%)
Date                                                             
1976-07-01                    7.83         -0.03           0.9225
1976-08-01                    7.77         -0.06           1.1875
1976-09-01                    7.59         -0.18           2.2525
1976-10-01                    7.41         -0.18           2.2375
1976-11-01                    7.29         -0.12           1.6875


In [66]:
import requests
import pandas as pd

# Your FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# TB3MS: 3-Month Treasury Bill Rate
series_id_cash = 'TB3MS'
units_cash = 'lin'                 # Raw yield in %
frequency_cash = 'm'               # Monthly frequency
start_date_cash = '1976-06-01'
end_date_cash = '2025-03-27'

# FRED API setup
url_cash = "https://api.stlouisfed.org/fred/series/observations"
params_cash = {
    'series_id': series_id_cash,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_cash,
    'frequency': frequency_cash,
    'observation_start': start_date_cash,
    'observation_end': end_date_cash
}

# Make the request
response_cash = requests.get(url_cash, params=params_cash)

# Process response
if response_cash.status_code == 200:
    data_cash = response_cash.json()['observations']
    df_cash = pd.DataFrame(data_cash)

    # Format and clean
    df_cash['date'] = pd.to_datetime(df_cash['date'])
    df_cash['value'] = pd.to_numeric(df_cash['value'], errors='coerce')
    df_cash = df_cash[['date', 'value']].dropna().set_index('date')
    df_cash.rename(columns={'value': '3M Treasury Yield (%)'}, inplace=True)

    # Estimate monthly return as yield / 12
    df_cash['Cash Return (%)'] = df_cash['3M Treasury Yield (%)'] / 12
    df_cash.index.name = "Date"

    # Preview
    print(df_cash.head())

else:
    print('Failed to retrieve 3M Treasury Bill data.')
    print('Status Code:', response_cash.status_code)
    print(response_cash.text)



            3M Treasury Yield (%)  Cash Return (%)
Date                                              
1976-06-01                   5.41         0.450833
1976-07-01                   5.23         0.435833
1976-08-01                   5.14         0.428333
1976-09-01                   5.08         0.423333
1976-10-01                   4.92         0.410000


In [67]:
import requests
import pandas as pd

# Your FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# IR14270: Gold Price (USD/oz)
series_id_gold = 'IR14270'
units_gold = 'lin'                 # Raw price
frequency_gold = 'm'               # Monthly
start_date_gold = '1976-06-01'
end_date_gold = '2025-03-27'

# FRED API setup
url_gold = "https://api.stlouisfed.org/fred/series/observations"
params_gold = {
    'series_id': series_id_gold,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_gold,
    'frequency': frequency_gold,
    'observation_start': start_date_gold,
    'observation_end': end_date_gold
}

# Make the request
response_gold = requests.get(url_gold, params=params_gold)

# Process response
if response_gold.status_code == 200:
    data_gold = response_gold.json()['observations']
    df_gold = pd.DataFrame(data_gold)

    # Format and clean
    df_gold['date'] = pd.to_datetime(df_gold['date'])
    df_gold['value'] = pd.to_numeric(df_gold['value'], errors='coerce')
    df_gold = df_gold[['date', 'value']].dropna().set_index('date')
    df_gold.rename(columns={'value': 'Gold Spot Price ($)'}, inplace=True)

    # Compute Month-over-Month return
    df_gold['Gold MoM Return (%)'] = df_gold['Gold Spot Price ($)'].pct_change() * 100
    df_gold = df_gold.dropna()
    df_gold.index.name = "Date"

    # Preview
    print(df_gold.head())

else:
    print('Failed to retrieve Gold data.')
    print('Status Code:', response_gold.status_code)
    print(response_gold.text)


            Gold Spot Price ($)  Gold MoM Return (%)
Date                                                
1993-03-01                121.1            -0.329218
1993-06-01                134.4            10.982659
1993-09-01                133.5            -0.669643
1993-12-01                135.6             1.573034
1994-01-01                140.8             3.834808


In [68]:
import requests
import pandas as pd

# FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# SPGSCI: S&P GSCI Commodity Index
series_id_commodities = 'SPGSCI'
units_commodities = 'lin'         # Raw price level
frequency_commodities = 'm'       # Monthly
start_date_commodities = '1976-06-01'
end_date_commodities = '2025-03-27'

# FRED API setup
url_commodities = "https://api.stlouisfed.org/fred/series/observations"
params_commodities = {
    'series_id': series_id_commodities,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_commodities,
    'frequency': frequency_commodities,
    'observation_start': start_date_commodities,
    'observation_end': end_date_commodities
}

# Make the request
response_commodities = requests.get(url_commodities, params=params_commodities)

# Process response
if response_commodities.status_code == 200:
    data_commodities = response_commodities.json()['observations']
    df_commodities = pd.DataFrame(data_commodities)

    # Clean and format
    df_commodities['date'] = pd.to_datetime(df_commodities['date'])
    df_commodities['value'] = pd.to_numeric(df_commodities['value'], errors='coerce')
    df_commodities = df_commodities[['date', 'value']].dropna().set_index('date')
    df_commodities.rename(columns={'value': 'GSCI Commodity Index'}, inplace=True)

    # Calculate Month-over-Month % change
    df_commodities['Commodities MoM Return (%)'] = df_commodities['GSCI Commodity Index'].pct_change() * 100
    df_commodities = df_commodities.dropna()
    df_commodities.index.name = "Date"

    # Preview
    print(df_commodities.head())

else:
    print('Failed to retrieve Commodities data.')
    print('Status Code:', response_commodities.status_code)
    print(response_commodities.text)


Failed to retrieve Commodities data.
Status Code: 400
{"error_code":400,"error_message":"Bad Request.  The series does not exist."}


In [69]:
import requests
import pandas as pd

# FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# PPIACO: Producer Price Index by Commodity: All Commodities
series_id_ppi = 'PPIACO'
units_ppi = 'lin'               # Raw index value
frequency_ppi = 'm'             # Monthly
start_date_ppi = '1976-06-01'
end_date_ppi = '2025-03-27'

# FRED API setup
url_ppi = "https://api.stlouisfed.org/fred/series/observations"
params_ppi = {
    'series_id': series_id_ppi,
    'api_key': api_key,
    'file_type': 'json',
    'units': units_ppi,
    'frequency': frequency_ppi,
    'observation_start': start_date_ppi,
    'observation_end': end_date_ppi
}

# Request and format
response_ppi = requests.get(url_ppi, params=params_ppi)

if response_ppi.status_code == 200:
    data_ppi = response_ppi.json()['observations']
    df_ppi = pd.DataFrame(data_ppi)

    df_ppi['date'] = pd.to_datetime(df_ppi['date'])
    df_ppi['value'] = pd.to_numeric(df_ppi['value'], errors='coerce')
    df_ppi = df_ppi[['date', 'value']].dropna().set_index('date')
    df_ppi.rename(columns={'value': 'PPI All Commodities Index'}, inplace=True)

    # Calculate % change
    df_ppi['PPI MoM Return (%)'] = df_ppi['PPI All Commodities Index'].pct_change() * 100
    df_ppi = df_ppi.dropna()

    print(df_ppi.head())

else:
    print('Failed to retrieve PPIACO data.')
    print('Status Code:', response_ppi.status_code)
    print(response_ppi.text)


            PPI All Commodities Index  PPI MoM Return (%)
date                                                     
1976-07-01                       61.6            0.653595
1976-08-01                       61.4           -0.324675
1976-09-01                       61.8            0.651466
1976-10-01                       61.9            0.161812
1976-11-01                       62.0            0.161551


In [70]:
import requests
import pandas as pd

# FRED API Key
api_key = 'fd0c3925530e1adec9762968a731218b'

# CPIAUCSL: Consumer Price Index (All Urban Consumers, All Items)
series_id_cpi = 'CPIAUCSL'
start_date_cpi = '1976-06-01'
end_date_cpi = '2025-03-27'

# FRED API URL
url_cpi = "https://api.stlouisfed.org/fred/series/observations"
params_cpi = {
    'series_id': series_id_cpi,
    'api_key': api_key,
    'file_type': 'json',
    'units': 'lin',
    'frequency': 'm',
    'observation_start': start_date_cpi,
    'observation_end': end_date_cpi
}

# Make the request
response_cpi = requests.get(url_cpi, params=params_cpi)

# Format response
if response_cpi.status_code == 200:
    data_cpi = response_cpi.json()['observations']
    df_cpi = pd.DataFrame(data_cpi)

    # Format and clean
    df_cpi['date'] = pd.to_datetime(df_cpi['date'])
    df_cpi['value'] = pd.to_numeric(df_cpi['value'], errors='coerce')
    df_cpi = df_cpi[['date', 'value']].dropna().set_index('date')
    df_cpi.rename(columns={'value': 'CPI Index'}, inplace=True)

    # Calculate MoM and YoY inflation
    df_cpi['CPI MoM (%)'] = df_cpi['CPI Index'].pct_change() * 100
    df_cpi['CPI YoY (%)'] = df_cpi['CPI Index'].pct_change(12) * 100
    df_cpi = df_cpi.dropna()
    df_cpi.index.name = "Date"

    # Preview
    print(df_cpi.head())

else:
    print('Failed to retrieve CPI data.')
    print('Status Code:', response_cpi.status_code)
    print(response_cpi.text)


            CPI Index  CPI MoM (%)  CPI YoY (%)
Date                                           
1977-06-01       60.5     0.498339     6.701940
1977-07-01       60.8     0.495868     6.666667
1977-08-01       61.1     0.493421     6.631763
1977-09-01       61.3     0.327332     6.423611
1977-10-01       61.6     0.489396     6.390328


In [71]:
# Merge all DataFrames on the Date index
df_asset_returns = pd.concat([
    df_sp500,
    df_bonds,
    df_cash,
    df_gold,
    df_ppi,
    df_cpi
], axis=1)

# Find the first date where ALL columns are non-null
first_valid_date = df_asset_returns.dropna().index.min()

# Keep only data from that date forward
df_asset_returns = df_asset_returns.loc[first_valid_date:]

# Forward fill from there (fills any quarterly gaps)
df_asset_returns = df_asset_returns.ffill()

# Format to 2 decimal places
for col in df_asset_returns.columns:
    df_asset_returns[col] = df_asset_returns[col].map(lambda x: f"{x:.2f}")

# Reset index and add Date column explicitly
df_asset_returns = df_asset_returns.reset_index()
df_asset_returns.rename(columns={'index': 'Date'}, inplace=True)

# Preview
print(df_asset_returns.head())


        Date S&P 500 MoM Return (%) 10Y Treasury Yield (%) Yield Change  \
0 1993-03-01                   1.93                   5.98        -0.28   
1 1993-04-01                  -1.37                   5.97        -0.01   
2 1993-05-01                   0.53                   6.04         0.07   
3 1993-06-01                   0.47                   5.96        -0.08   
4 1993-07-01                   0.28                   5.81        -0.15   

  Bond Return (%) 3M Treasury Yield (%) Cash Return (%) Gold Spot Price ($)  \
0            3.02                  2.95            0.25              121.10   
1            0.59                  2.87            0.24              121.10   
2           -0.13                  2.96            0.25              121.10   
3            1.22                  3.07            0.26              134.40   
4            1.83                  3.04            0.25              134.40   

  Gold MoM Return (%) PPI All Commodities Index PPI MoM Return (%) CPI Ind

In [72]:
# Desired column order
ordered_columns = [
    "Date",
    "CPI MoM (%)",
    "S&P 500 MoM Return (%)",
    "Bond Return (%)",
    "Cash Return (%)",
    "Gold MoM Return (%)",
    "PPI MoM Return (%)"
]

# Reorder the DataFrame
df_monthly_returns = df_asset_returns[ordered_columns]

print(df_monthly_returns.head())


        Date CPI MoM (%) S&P 500 MoM Return (%) Bond Return (%)  \
0 1993-03-01        0.14                   1.93            3.02   
1 1993-04-01        0.35                  -1.37            0.59   
2 1993-05-01        0.28                   0.53           -0.13   
3 1993-06-01        0.07                   0.47            1.22   
4 1993-07-01        0.14                   0.28            1.83   

  Cash Return (%) Gold MoM Return (%) PPI MoM Return (%)  
0            0.25               -0.33               0.25  
1            0.24               -0.33               0.51  
2            0.25               -0.33               0.34  
3            0.26               10.98              -0.17  
4            0.25               10.98              -0.25  


In [95]:
# 1. Create a copy to avoid SettingWithCopyWarning
df_monthly_returns = df_monthly_returns.copy()

# 2. Define only asset return columns
return_columns = [
    "S&P 500 MoM Return (%)",
    "Bond Return (%)",
    "Cash Return (%)",
    "Gold MoM Return (%)",
    "PPI MoM Return (%)"
]

# 3. Ensure return columns are numeric
df_monthly_returns[return_columns] = df_monthly_returns[return_columns].apply(pd.to_numeric, errors="coerce")

# 4. Identify best and worst performing assets
df_monthly_returns["Best Asset"] = df_monthly_returns[return_columns].idxmax(axis=1)
df_monthly_returns["Worst Asset"] = df_monthly_returns[return_columns].idxmin(axis=1)

# 5. Use apply and row indexing to get the best/worst return values
df_monthly_returns["Best Return (%)"] = df_monthly_returns.apply(
    lambda row: row[row["Best Asset"]], axis=1
)

df_monthly_returns["Worst Return (%)"] = df_monthly_returns.apply(
    lambda row: row[row["Worst Asset"]], axis=1
)


df_monthly_returns["Best Return (%)"] = df_monthly_returns["Best Return (%)"].map(lambda x: f"{x:.2f}%")
df_monthly_returns["Worst Return (%)"] = df_monthly_returns["Worst Return (%)"].map(lambda x: f"{x:.2f}%")


# ✅ Preview final output
print(df_monthly_returns[["Date", "Best Asset", "Best Return (%)", "Worst Asset", "Worst Return (%)"]].head())




        Date              Best Asset Best Return (%)             Worst Asset  \
0 1993-03-01         Bond Return (%)           3.02%     Gold MoM Return (%)   
1 1993-04-01         Bond Return (%)           0.59%  S&P 500 MoM Return (%)   
2 1993-05-01  S&P 500 MoM Return (%)           0.53%     Gold MoM Return (%)   
3 1993-06-01     Gold MoM Return (%)          10.98%      PPI MoM Return (%)   
4 1993-07-01     Gold MoM Return (%)          10.98%      PPI MoM Return (%)   

  Worst Return (%)  
0           -0.33%  
1           -1.37%  
2           -0.33%  
3           -0.17%  
4           -0.25%  


In [97]:
df_monthly_returns.to_csv("df_monthly_returns.csv", index = False)

In [104]:
import pandas as pd
import numpy as np

# Load both datasets
df_returns = pd.read_csv("df_monthly_returns.csv", parse_dates=["Date"])
df_regimes = pd.read_csv("regime_model_labeled_percentage.csv", parse_dates=["Date"])

# Merge on date
df_merged = pd.merge(df_returns, df_regimes[["Date", "Regime_Label"]], on="Date", how="inner")

# Define asset return columns
return_columns = [
    "S&P 500 MoM Return (%)",
    "Bond Return (%)",
    "Cash Return (%)",
    "Gold MoM Return (%)",
    "PPI MoM Return (%)"
]

# Ensure return columns are numeric
df_merged[return_columns] = df_merged[return_columns].apply(pd.to_numeric, errors="coerce")

# Find best and worst assets
df_merged["Best Asset"] = df_merged[return_columns].idxmax(axis=1)
df_merged["Worst Asset"] = df_merged[return_columns].idxmin(axis=1)

# Convert return columns to NumPy array for row-wise indexing
returns_array = df_merged[return_columns].to_numpy()
col_index = {col: i for i, col in enumerate(return_columns)}

# Get best and worst return values based on their labels
best_indices = [col_index[col] for col in df_merged["Best Asset"]]
worst_indices = [col_index[col] for col in df_merged["Worst Asset"]]
rows = np.arange(len(df_merged))

df_merged["Best Return (%)"] = returns_array[rows, best_indices]
df_merged["Worst Return (%)"] = returns_array[rows, worst_indices]

# Round values
df_merged["Best Return (%)"] = df_merged["Best Return (%)"].round(2)
df_merged["Worst Return (%)"] = df_merged["Worst Return (%)"].round(2)

print("✅ Final CSV created: 'merged_regimes_returns_cleaned.csv'")



✅ Final CSV created: 'merged_regimes_returns_cleaned.csv'


In [106]:

# Save to CSV
df_merged.to_csv("merged_regimes_returns_cleaned.csv", index=False)


In [111]:
print(df_merged.groupby("Regime_Label")["Best Asset"].value_counts(normalize=True).unstack().fillna(0).round(2))


Best Asset      Bond Return (%)  Cash Return (%)  Gold MoM Return (%)  \
Regime_Label                                                            
Crisis                     0.14             0.09                 0.23   
Early_Recovery             0.23             0.02                 0.33   
Expansion                  0.26             0.04                 0.37   
Late_Cycle                 0.21             0.04                 0.37   
Shock_Recovery             0.17             0.00                 0.17   
Stagnation                 0.19             0.05                 0.23   

Best Asset      PPI MoM Return (%)  S&P 500 MoM Return (%)  
Regime_Label                                                
Crisis                        0.23                    0.32  
Early_Recovery                0.08                    0.34  
Expansion                     0.04                    0.30  
Late_Cycle                    0.08                    0.29  
Shock_Recovery                0.42               

In [113]:
print(df_merged.groupby("Regime_Label")["Worst Asset"].value_counts(normalize=True).unstack().fillna(0).round(2))


Worst Asset     Bond Return (%)  Cash Return (%)  Gold MoM Return (%)  \
Regime_Label                                                            
Crisis                     0.27             0.05                 0.32   
Early_Recovery             0.14             0.08                 0.37   
Expansion                  0.19             0.15                 0.30   
Late_Cycle                 0.21             0.12                 0.24   
Shock_Recovery             0.17             0.25                 0.42   
Stagnation                 0.26             0.11                 0.33   

Worst Asset     PPI MoM Return (%)  S&P 500 MoM Return (%)  
Regime_Label                                                
Crisis                        0.14                    0.23  
Early_Recovery                0.09                    0.32  
Expansion                     0.11                    0.26  
Late_Cycle                    0.19                    0.24  
Shock_Recovery                0.00               

In [115]:
print(df_merged.groupby("Regime_Label")[return_columns].mean().round(2))


                S&P 500 MoM Return (%)  Bond Return (%)  Cash Return (%)  \
Regime_Label                                                               
Crisis                            0.25            -0.74             0.31   
Early_Recovery                    0.10             0.42             0.07   
Expansion                        -0.08             0.62             0.02   
Late_Cycle                        0.68             0.71             0.33   
Shock_Recovery                    1.10            -0.38             0.00   
Stagnation                        0.97             0.33             0.25   

                Gold MoM Return (%)  PPI MoM Return (%)  
Regime_Label                                             
Crisis                         0.18                0.49  
Early_Recovery                 0.47                0.12  
Expansion                      1.33               -0.36  
Late_Cycle                     1.64                0.26  
Shock_Recovery                -0.46        

In [117]:
# Average the returns grouped by regime
df_avg_returns = df_merged.groupby("Regime_Label")[
    [
        "S&P 500 MoM Return (%)",
        "Bond Return (%)",
        "Cash Return (%)",
        "Gold MoM Return (%)",
        "PPI MoM Return (%)"
    ]
].mean().round(2)

# Count how often each asset was the best in each regime
df_best_count = df_merged.groupby("Regime_Label")["Best Asset"].value_counts().unstack().fillna(0).astype(int)

# Combine both into one display (optional)
print("📊 Average Returns by Regime:")
print(df_avg_returns)

print("\n🏆 Frequency of Best-Performing Asset by Regime:")
print(df_best_count)


📊 Average Returns by Regime:
                S&P 500 MoM Return (%)  Bond Return (%)  Cash Return (%)  \
Regime_Label                                                               
Crisis                            0.25            -0.74             0.31   
Early_Recovery                    0.10             0.42             0.07   
Expansion                        -0.08             0.62             0.02   
Late_Cycle                        0.68             0.71             0.33   
Shock_Recovery                    1.10            -0.38             0.00   
Stagnation                        0.97             0.33             0.25   

                Gold MoM Return (%)  PPI MoM Return (%)  
Regime_Label                                             
Crisis                         0.18                0.49  
Early_Recovery                 0.47                0.12  
Expansion                      1.33               -0.36  
Late_Cycle                     1.64                0.26  
Shock_Recovery

In [119]:
df_merged.to_csv("regime_analysis_results.csv", index=False)
